In [3]:
#8 hour horizon model
import cvxpy as cp
import numpy as np

i = 8 #horizon
lambda_ = 1
l = 0.75 #soft constraint
n = 8
T_min = 40
T_max = 60

#example value
#model

M = np.array([[6.25460712e-03, -4.17892614e-02,  1.45078700e-02,
        -1.48634945e-04, -1.65626542e-03, -3.98750475e-03,
        -4.01025626e-03, -2.94109852e-03,  5.57711947e-02,
        -4.12871628e-03, -1.92634527e-02,  2.89880654e-02,
         1.03839646e-02,  4.27543933e-03, -2.30155374e-02,
         3.62389037e-02,  8.88731644e-01, -5.14401545e-03],
       [ 2.77638753e-02, -1.05860217e-01, -2.05706913e-02,
         1.18870177e-02, -3.16097290e-04, -2.91305429e-03,
        -4.44793805e-03, -2.90759753e-03,  5.82682483e-02,
         8.68213266e-03, -1.62680675e-03,  1.01281922e-02,
         2.56916233e-02, -3.15834816e-03, -2.10446746e-02,
         3.55276442e-02,  8.62335223e-01, -1.88669920e-03],
       [ 1.51693643e-02, -9.01459209e-02, -8.54562993e-02,
        -2.29734523e-02,  1.15367862e-02, -1.38193182e-03,
        -2.81366406e-03, -3.30682336e-03,  7.02235590e-02,
        -5.10075210e-03,  2.07881191e-02,  2.28335826e-02,
         1.54431650e-02,  3.10435917e-03, -2.40048971e-02,
         3.66494045e-02,  8.33101551e-01, -6.40011120e-04],
       [ 1.45554257e-02, -1.02992719e-01, -7.26927208e-02,
        -8.71559240e-02, -2.41501580e-02,  1.04887660e-02,
        -9.67297381e-04, -1.65804673e-03,  6.88879367e-02,
         9.19088439e-03,  5.36748804e-03,  4.70129494e-02,
         2.61752238e-02, -5.04828100e-03, -1.96994873e-02,
         3.59765419e-02,  8.04262858e-01, -6.13514419e-04],
       [ 1.59764637e-02, -1.01098912e-01, -8.60682054e-02,
        -7.30539695e-02, -8.91306653e-02, -2.41711655e-02,
         9.82865577e-03,  9.01233712e-04,  7.23084279e-02,
         1.39040304e-03,  2.46988117e-02,  2.73744080e-02,
         5.47915173e-02,  1.55808149e-03, -2.48843906e-02,
         3.84811029e-02,  7.76249087e-01, -1.34175681e-03],
       [ 1.09900244e-02, -9.31684910e-02, -8.61747063e-02,
        -8.74173110e-02, -7.58464384e-02, -8.87446600e-02,
        -2.55574887e-02,  1.35911660e-02,  7.13763518e-02,
         5.28459399e-03,  1.57597437e-02,  4.81736747e-02,
         3.37768118e-02,  3.18487505e-02, -2.12764775e-02,
         3.71605419e-02,  7.50147713e-01, -2.54332914e-03],
       [ 1.18276710e-02, -8.88973975e-02, -7.75248347e-02,
        -8.69995610e-02, -8.98479955e-02, -7.42300517e-02,
        -9.07444324e-02, -1.85131922e-02,  7.31369099e-02,
        -4.88046871e-04,  2.37528100e-02,  3.57408749e-02,
         5.82218647e-02,  7.59767931e-03,  1.14982613e-02,
         3.83875961e-02,  7.25159239e-01, -4.14010272e-03],
       [ 2.61027990e-02, -8.31925185e-02, -7.55482374e-02,
        -8.09662976e-02, -8.88863543e-02, -8.95595424e-02,
        -7.73536288e-02, -9.23762527e-02,  6.82204451e-02,
         7.10422024e-03,  1.44099871e-02,  4.60600721e-02,
         4.21294528e-02,  3.60398764e-02, -1.55777123e-02,
         7.32810268e-02,  7.01950982e-01, -5.62962301e-03]]
)


N = [[*ele[8:16]] for ele in M] #Setpoints
N = np.array(N)
M_new = [[*x[0:8], *x[-2:]] for x in M]

M = np.array(M_new)

def getOptimalSetpoint(x): #input values [d1...d8, twh_3,]

  # Construct the problem.
  #declare variables
  T = cp.Variable(i)
  s = cp.Variable(i) #setpoint variables
  z = cp.Variable(i)
  objective = cp.Minimize(cp.sum(lambda_ * z**2 + (lambda_*l) * s))

  constraints = [
      s >= 49,
      s <= T_max,
      T == M @ x + N @ s,
      z >= T_min - T,
      z >= 0
      ]
  prob = cp.Problem(objective, constraints)

  # The optimal objective value is returned by `prob.solve()`.
  result = prob.solve()

  # Print results
  print("Optimal T values:", T.value)
  print("Optimal s values:", s.value)
  print("Optimal objective value:", result)

  return s

In [9]:
x = [0, 50, 0, 50, 0, 50, 0,  0, 39.41643795734582,39.41643795734583]
getOptimalSetpoint(x)

Optimal T values: [38.18518536 35.9555718  35.80395794 32.88139063 32.62941992 29.50352889
 30.66342014 31.15384023]
Optimal s values: [60. 49. 60. 60. 60. 60. 49. 60.]
Optimal objective value: 761.3599427138514


Variable((8,), var362)

In [ ]:
#Query Ochre
import os
import datetime as dt
import pandas as pd
import numpy as np

from ochre import Dwelling
from ochre.utils import default_input_path  # for using sample files
from ochre import HeatPumpWaterHeater



def simulate_ochre():
    # Define equipment and simulation parameters
    setpoint_default = 51  # in C
    deadband_default = 5.56  # in C
    max_setpoint = 60
    min_setpoint = 40

    run_range = False #runs simulation for a variety of setpoints specified in setpoint_range
    simulation_days = 1 #172 #220
    site_number = 90023#10292#'10441'

    site_number = 'null'

    flow_data = f'net_flow_{site_number}.csv'

    #start_date = dt.datetime(2013, 1, 17, 0, 1) #10441
    start_date = dt.datetime(2013, 1, 1, 0, 1) #10292, 90023

    setpoint_range = [setpoint_default]

    if run_range == True:
        setpoint_range = np.arange(min_setpoint, max_setpoint, 0.5)

    x = [0, 0, 0, 0, 0, 0, 0, 0, 51, 51]
    setpoint_pred = getOptimalSetpoint(x)

    for s in setpoint_range: #run simulation for every setpoint in valid range
        setpoint_default = s
        print("Simulating Setpoint: ", setpoint_pred)
        equipment_args = {
            "start_time": start_date,  # year, month, day, hour, minute
            "time_res": dt.timedelta(minutes=1),
            "duration": dt.timedelta(minutes=120),
            "verbosity": 9,  # required to get setpoint and deadband in results
            "save_results": False,  # if True, must specify output_path
            # "output_path": os.getcwd(),        # Equipment parameters
            "Setpoint Temperature (C)": setpoint_default,
            "Tank Volume (L)": 250,
            "Tank Height (m)": 1.22,
            "UA (W/K)": 2.17,
            "HPWH COP (-)": 4.5,
        }

        # Create water draw schedule
        times = pd.date_range(
            equipment_args["start_time"],
            equipment_args["start_time"] + equipment_args["duration"],
            freq=equipment_args["time_res"],
            inclusive="left",
        )
        water_draw_magnitude = 12  # L/min
        #withdraw_rate = np.random.choice([0, water_draw_magnitude], p=[0.99, 0.01], size=len(times))
        withdraw_rate = np.loadtxt(f'ochre\defaults\\Input Files\\{flow_data}')
        withdraw_rate = withdraw_rate[:len(times)]
        schedule = pd.DataFrame(
            {
                "Water Heating (L/min)": withdraw_rate,
                "Water Heating Setpoint (C)": setpoint_default,  # Setting so that it can reset
                "Water Heating Deadband (C)": deadband_default,  # Setting so that it can reset
                "Zone Temperature (C)": 20,
                "Zone Wet Bulb Temperature (C)": 15,  # Required for HPWH
                "Mains Temperature (C)": 7,
            },
            index=times,
        )

        # Initialize equipment
        hpwh = HeatPumpWaterHeater(schedule=schedule, **equipment_args)

        # Simulate
        data = pd.DataFrame()
        data = {'draw_data' :[], 'setpoint' :[]}
        control_signal = {}
        setpoints = []

        #generate noise for setpoint profile
        noise = np.random.randint(0, 3, 24, dtype=np.int8)
        for t in hpwh.sim_times:
            # Change setpoint based on hour of day
            setpoint = setpoint_default

            setpoint_pred = getOptimalSetpoint(x)
            control_signal = {
                "Setpoint": setpoint_pred[0]
            }
            setpoints.append(setpoint)
            # Run with controls
            _ = hpwh.update(control_signal=control_signal)

        df = hpwh.finalize()

        cols_to_plot = [
            "Hot Water Outlet Temperature (C)",
            "Hot Water Average Temperature (C)",
            "Water Heating Deadband Upper Limit (C)",
            "Water Heating Deadband Lower Limit (C)",
            "Water Heating Electric Power (kW)",
            "Hot Water Unmet Demand (kW)",
            "Hot Water Delivered (L/min)",
        ]

        cols_to_save = [
            "Hot Water Outlet Temperature (C)",
            "T_WH3",
            "T_WH10"
        ]


        withdraw_rate = np.convolve(withdraw_rate, np.ones(15)/15, 'same')

        # Calculate the rolling average for 'setpoints' with window size 15
        setpoints = np.convolve(setpoints, np.ones(15)/15, 'same')

        # For the DataFrame, select columns and calculate the rolling average for each column
        to_save = df[cols_to_save].rolling(window=15).mean()

        draw_data = withdraw_rate[14::15]
        setpoints = setpoints[14::15]
        to_save = df.loc[:, cols_to_save]
        to_save = to_save[14::15]

        to_save["Draw Data"] = pd.Series(draw_data, index=to_save.index)
        to_save["Setpoint"] = pd.Series(setpoints, index=to_save.index)

        print(len(draw_data), len(setpoints), len(to_save))

        import matplotlib.pyplot as plt 

        df.loc[:, cols_to_plot].plot()
        print("Simulation Copmleted")
        return cols_to_save
        #to_save.to_csv(f'output_site_{site_number}.csv', mode='a', header=False, index=False)

    #plt.show()



In [3]:
#prbs
def prbs31(code, n):
    for i in range(n):
        next_bit = ~((code>>30) ^ (code>>27))&0x01
        code = ((code<<1) | next_bit) & 0xFFFFFFFF
    return code

In [1]:
import pandas as pd
import numpy as np
T = pd.read_csv("output_site_90159.csv")
flow = T["Draw Data (L)"].unique()

T = pd.read_csv("output_site_90023.csv")
heavy_draw = T["Draw Data (L)"].unique()
heavy_draw = np.concatenate((heavy_draw, flow))
heavy_draw.sort()

In [38]:
heavy_draw

array([ 0.        ,  0.        ,  0.25236067,  0.50472133,  0.757082  ,
        0.95528859,  1.00944267,  1.26180333,  1.514164  ,  1.76652467,
        1.91057718,  2.01888533,  2.271246  ,  2.52360667,  2.77596733,
        2.86586577,  3.028328  ,  3.28068867,  3.53304933,  3.78541   ,
        3.82115436,  4.03777067,  4.29013133,  4.542492  ,  4.77644296,
        4.79485267,  5.04721333,  5.299574  ,  5.55193467,  5.73173155,
        5.80429533,  6.056656  ,  6.30901667,  6.56137733,  6.68702014,
        6.68702014,  6.813738  ,  7.06609867,  7.31845933,  7.57082   ,
        7.64230873,  7.82318067,  8.58026267,  8.59759732,  8.59759732,
        9.55288591,  9.55288591,  9.58970533, 10.5081745 , 10.85150867,
       11.46346309, 12.41875169, 13.37404028, 13.37404028, 14.32932887,
       14.32932887, 15.28461746, 15.28461746, 15.28461746, 16.23990605,
       16.23990605, 17.19519464, 17.19519464, 18.15048323, 18.15048323,
       19.10577182, 19.10577182, 20.06106042, 21.01634901, 21.97

In [35]:
len(heavy_draw)
prob = np.random.randint(1, 100, len(heavy_draw))
prob[0] = 250000
prob = prob/ sum(prob)
prob

array([9.83438889e-01, 2.98965422e-04, 2.98965422e-04, 2.91097911e-04,
       2.04555289e-04, 2.28157822e-04, 2.39959089e-04, 5.11388222e-05,
       6.29400889e-05, 1.37681444e-04, 3.89441800e-04, 9.83438889e-05,
       2.75362889e-04, 1.96687778e-04, 2.98965422e-04, 3.38302978e-04,
       1.88820267e-04, 3.85508045e-04, 6.29400889e-05, 3.42236733e-04,
       3.65839267e-04, 4.32713111e-05, 1.92754022e-04, 1.61283978e-04,
       7.86751111e-05, 7.86751111e-05, 2.12422800e-04, 1.69151489e-04,
       2.79296645e-04, 3.77640533e-04, 3.34369222e-04, 1.37681444e-04,
       1.57350222e-04, 2.36025333e-05, 3.30435467e-04, 3.93375556e-05,
       3.18634200e-04, 3.34369222e-04, 1.14078911e-04, 3.54038000e-05,
       3.93375556e-05, 3.30435467e-04, 3.34369222e-04, 1.41615200e-04,
       3.50104245e-04, 6.68738445e-05, 3.34369222e-04, 3.54038000e-05,
       1.21946422e-04, 2.75362889e-05, 1.96687778e-04, 9.04763778e-05,
       3.57971756e-04, 2.75362889e-05, 1.77019000e-04, 3.54038000e-05,
      

In [ ]:

draws = np.random.choice(heavy_draw, 844784, p = prob)
draws = np.asarray(draws)
np.savetxt('large_draw.csv', draws)

In [17]:
import random
#Generate heavy draw data, desired length is 844784
draw = []
draw_budget = 0
for i in range(844784):
    random_int = random.randint(1, 100)
    if i % 1440 == 0:
        print(draw_budget)
        draw_budget = 0
    if random_int >= 94 and draw_budget < 50:
        draw_value = np.random.choice(heavy_draw, 1)[0]
        print(draw_value)
        draw.append(draw_value)
        draw_budget += draw_value
    else:
        draw.append(0)

np.savetxt('net_flow_large.csv', draw)

0
0.2523606666666667
2.775967333333333
3.028328
29.613946327406676
4.542492
0.9552885912066668
13.374040276893336
54.542423195506686
21.971637597753336
1.514164
26.74808055378667
50.23388215154001
16.239906050513337
7.066098666666667
2.5236066666666668
28.65865773620001
54.48826912004668
24.837503371373337
1.0094426666666667
30.569234918613336
56.416180956653335
3.78541
34.39038928344
4.542492
17.195194641720004
59.91348592516
9.589705333333336
2.0188853333333334
7.57082
2.5236066666666668
33.435100692233334
55.13811802556667
7.823180666666667
5.551934666666667
3.533049333333333
7.318459333333333
13.374040276893334
31.524523509820007
69.12518778671334
9.55288591206667
17.19519464172
3.533049333333333
7.823180666666667
32.47981210102667
70.58412265481334
4.290131333333333
7.57082
22.926926188960007
1.2618033333333334
42.03269801309334
78.08237886872001
5.0472133333333336
4.794852666666667
22.926926188960007
0.2523606666666667
16.239906050513337
25.79279196258
75.05405086872001
6.3090166

In [11]:
draw

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3.280688666666667,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 42.03269801309334,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 19.10577182413334,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
